# Primeiros Passos na Segmentação de Imagens Ortomosaicas

## 1. Segmentação das Imagens
O primeiro passo consiste em dividir as imagens ortomosaicas em várias imagens menores de 600x600 pixels, separando todas as camadas (R, G, B) individualmente. Essa etapa é essencial para alimentar o modelo corretamente e otimizar o processamento das imagens.

## 2. Importação de Modelos Pré-treinados e Dataset
Após a segmentação, é necessário importar os modelos pré-treinados e o dataset para realizar inferências (predições). Utilizaremos o modelo **DeepLabV3Plus** com **ResNet-50** treinado para classificação de três classes: **fundo, milho e daninha**.

Repositório do modelo: [DeepLabV3Plus-PyTorch](https://github.com/VainF/DeepLabV3Plus-Pytorch)

### 2.1 Análise e Recorte do Dataset
Aqui entra uma parte mais complexa: precisamos analisar o dataset segmentado anteriormente e garantir que nossas imagens estejam devidamente rotuladas. Ferramentas úteis para essa etapa incluem:
- **LabelMe**
- **CUAT**
- **VGG Image Annotator (VIA)**

Essas ferramentas ajudam a criar e validar anotações para treinamento do modelo.

## 3. Fine-tuning ou Transfer Learning
Com as imagens devidamente segmentadas e anotadas, podemos prosseguir para a etapa de **fine-tuning** ou **transfer learning**. Esse processo ajusta o modelo pré-treinado para obter melhor desempenho na segmentação de milho e plantas daninhas, refinando os pesos da rede neural com nosso próprio dataset.

Essa abordagem permite que o modelo se adapte às características específicas das nossas imagens, melhorando sua precisão na segmentação.




* **1** Para atingir esse objetivo, usei Python juntamente com a biblioteca Pillow, que captura uma a uma das imagens .tif e as recorta em blocos de 600x600. Em seguida, ignora as imagens com mais de 10% de alpha e as separa nas camadas RGB, gerando os arquivos nome_imagemR.png, nome_imagemG.png, nome_imagemB.png. As imagens .tif estão localizadas no diretório /imgs/tifs, e as imagens .png são salvas em /imgs/pngs.

In [1]:
#Atualizar dependencias
!pip freeze > requirements.txt

In [ ]:
#Instalar dependencias 
!pip install -r requirements.txt

In [ ]:
import os
from PIL import Image
import numpy as np

# Diretórios
Image.MAX_IMAGE_PIXELS = None
input_dir = './imgs/tifs'  # Diretório com as imagens .tif
output_dir = './imgs/pngs'  # Diretório para salvar as imagens .png

# Função para verificar o alpha da imagem
def has_high_alpha(img, threshold=0.1):
    # Obtém o canal alpha, se existir
    alpha = np.array(img.split()[3]) if img.mode == 'RGBA' else None
    if alpha is not None:
        # Calcula a porcentagem de pixels com alpha > 10%
        alpha_percentage = np.sum(alpha > 25) / alpha.size  # Ajuste o valor 25 conforme necessário
        return alpha_percentage > threshold
    return False

# Função para salvar as imagens RGB separadas
# Função para salvar as imagens RGB separadas
def save_rgb_layers(img, base_name):
    # Verifica o modo da imagem
    if img.mode == 'RGB':
        r, g, b = img.split()  # Divide a imagem em suas camadas RGB
    elif img.mode == 'RGBA':
        r, g, b, a = img.split()  # Se a imagem tem canal alpha, divide também
    else:
        # Caso a imagem tenha outro modo, não processa (por exemplo, 'L' para escala de cinza)
        print(f"Imagem no modo {img.mode} não é suportada para divisão RGB.")
        return

    # Converte as camadas para o modo correto antes de salvar
    r = r.convert('RGB')
    g = g.convert('RGB')
    b = b.convert('RGB')

    # Salva cada camada como uma imagem separada
    r.save(os.path.join(output_dir, f"{base_name}R.png"))
    g.save(os.path.join(output_dir, f"{base_name}G.png"))
    b.save(os.path.join(output_dir, f"{base_name}B.png"))
    



# Função para processar as imagens
def process_images():
    # Cria o diretório de saída se não existir
    os.makedirs(output_dir, exist_ok=True)

    # Itera sobre todas as imagens no diretório de entrada
    qtdImgs = len(os.listdir(input_dir))
    for atual,filename in enumerate(os.listdir(input_dir)):
        if filename.endswith('.tif'):
            file_path = os.path.join(input_dir, filename)
            img = Image.open(file_path)
            
            # Processa a imagem em blocos de 600x600
            partesAproveitadas = 0
            partesPerdidas = 0
            img_width, img_height = img.size
            total = len(list(range(0, img_width, 600))) * len(list(range(0, img_height, 600)))
            for eixoX, i in enumerate(range(0, img_width, 600)):
                for eixoY, j in enumerate(range(0, img_height, 600)):
                    # Define as coordenadas do bloco
                    box = (i, j, min(i + 600, img_width), min(j + 600, img_height))
                    cropped_img = img.crop(box)
                    
                    # Cria um nome base para o arquivo
                    base_name = f"{filename.split('.')[0]}_{i}_{j}_"
                    
                    # Verifica se a parte da imagem tem alta transparência
                    if not has_high_alpha(cropped_img):
                        partesPerdidas += 1
                    else:
                        # Salva as camadas RGB separadas
                        save_rgb_layers(cropped_img, f'{base_name}{eixoX}X{eixoY}')
                        partesAproveitadas += 1
                    
                    # Exibe o progresso a cada iteração
                    progresso = ((eixoX * len(list(range(0, img_height, 600))) + eixoY) / total) * 100
                    print(f'Progresso {eixoX * len(list(range(0, img_height, 600))) + eixoY}/{total} --- {progresso:.2f}% --- Imagen {atual+1}/{qtdImgs}')

            print(f"Foram aproveitadas um total de {partesAproveitadas} imagens e {partesPerdidas} foram perdidas devido ao alto alpha.")

# Executa o processamento
if __name__ == "__main__":
    process_images()


Progresso 0/3596 --- 0.00% --- Imagen 1/5
Progresso 1/3596 --- 0.03% --- Imagen 1/5
Progresso 2/3596 --- 0.06% --- Imagen 1/5
Progresso 3/3596 --- 0.08% --- Imagen 1/5
Progresso 4/3596 --- 0.11% --- Imagen 1/5
Progresso 5/3596 --- 0.14% --- Imagen 1/5
Progresso 6/3596 --- 0.17% --- Imagen 1/5
Progresso 7/3596 --- 0.19% --- Imagen 1/5
Progresso 8/3596 --- 0.22% --- Imagen 1/5
Progresso 9/3596 --- 0.25% --- Imagen 1/5
Progresso 10/3596 --- 0.28% --- Imagen 1/5
Progresso 11/3596 --- 0.31% --- Imagen 1/5
Progresso 12/3596 --- 0.33% --- Imagen 1/5
Progresso 13/3596 --- 0.36% --- Imagen 1/5
Progresso 14/3596 --- 0.39% --- Imagen 1/5
Progresso 15/3596 --- 0.42% --- Imagen 1/5
Progresso 16/3596 --- 0.44% --- Imagen 1/5
Progresso 17/3596 --- 0.47% --- Imagen 1/5
Progresso 18/3596 --- 0.50% --- Imagen 1/5
Progresso 19/3596 --- 0.53% --- Imagen 1/5
Progresso 20/3596 --- 0.56% --- Imagen 1/5
Progresso 21/3596 --- 0.58% --- Imagen 1/5
Progresso 22/3596 --- 0.61% --- Imagen 1/5
Progresso 23/3596 ---

* **2** Como descrito acima, agora vamos importar os modelos pré-treinados [linkModelos](https://www.dropbox.com/scl/fo/i4k86qqiuzy33aawug287/ABz2JiKdDqjwvDDzMHpeKZI?rlkey=fgm7rgpeankeh9394j492sxif&e=1&dl=0), "Este mesmo link e disponibilizado pelo proprio git do DeepLabV3", logo tentar realizar uma inferência a partir deles. O passo a passo e exatamente o mesmo do descrito no proprio repositorio

In [ ]:
import torch
import network.modeling  
MODEL_NAME = 'deeplabv3_plus_resnet50'  
NUM_CLASSES = 21
OUTPUT_STRIDE = 16
PATH_TO_PTH = './modelos/best_deeplabv3plus_resnet50_voc_os16.pth' 
model = network.modeling.__dict__[MODEL_NAME](num_classes=NUM_CLASSES, output_stride=OUTPUT_STRIDE)
model.load_state_dict(torch.load(PATH_TO_PTH)['model_state'])

* **2.1** Com base nisso agr fica mais facil usar o [weedsgalore](https://github.com/GFZ/weedsgalore), rede baseada no [DeepLabV3Plus-PyTorch](https://github.com/VainF/DeepLabV3Plus-Pytorch)